# California PPE Projections 
## Editable View
This document provides direct access to the underlying data informing the projections. In addition to using sliders to adjust stockpile days, you can edit burn rate assumptions and rerun the model on the spot.

We provide detailed annotations of how to adjust model parameters for readers that might not have experience in programming or the Python programming language. However, if you do have a programming background, we expose our source code here for even more interactivity and editability. Simply use the button below to toggle on/off the raw code.

This is an alpha version of this view, so there are a couple extra steps to get started:

1. In the menu above, click on the `Kernel` button. A drop-down menu will appear; click `Restart and Run All`.
2. This will trigger a prompt asking you if you want to restart the kernel and re-run the whole notebook. Click the red button, `Restart and Run All Cells`.

At this point, setup is complete.

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import os
import sys
import qgrid
import pathlib
import restart
import numpy as np
import pandas as pd
from ipywidgets import widgets
from restart import RestartModel
from restart import Data
from restart_datasets import data
from IPython.display import Javascript, display
from restart import util
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells
from restart_datasets import data
import ipywidgets as widgets
import bqplot
from bqplot import pyplot as plt

qgrid.set_grid_option('forceFitColumns', False)
PATH = pathlib.Path(restart.__file__).parent.absolute()


def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, \
                        IPython.notebook.ncells())'))
    
def create_run_button(description):
    button = widgets.Button(description=description)
    button.on_click(run_all)
    return button

config = set_config(os.path.join(PATH, 'config/ca-vent'))
restart = RestartModel(config='ca-vent', population='dict')
model = restart.model

burn_df = model.demand.demand_per_unit_map_dn_um.df
burn_df.index.name = "Population"

burn_rates = qgrid.show_grid(burn_df)

# California Memo Burn Rates
Click on any of the cells below to adjust burn rate assumptions. When you're done, click the `Run Model` button and the updated model will be generated below (you'll have to scroll down a bit to see it). 

In [3]:
display(create_run_button("Run Model"))
widgets.VBox([burn_rates])

Button(description='Run Model', style=ButtonStyle())

In [4]:
import ipywidgets as widgets
from scipy.stats import norm
import math

# get updated burn rates
model.demand.adjust_burn(burn_rates.get_changed_df().to_numpy())

# charting 
burn_chart = util.generate_group_bar_legend(model.demand.demand_per_unit_map_dn_um.df)

ca_epi_df = data.epi_ranges()
ca_epi_df.set_index('Model', drop=True, inplace=True)

ca_epi_df = ca_epi_df * 4.12
ca_epi_df.columns = ['Population Low', 'Population Mid', 'Population High']
ca_stdev = ca_epi_df.apply(
    lambda row: util.triangular(
        row['Population Mid'],
        row['Population Low'], 
        row['Population High']
    ), axis=1
)

ca_epi_df['Population Mean'] = ca_epi_df.mean(axis=1)
ca_epi_df['Population SD'] = ca_stdev


def calc_eoq(df, cr):
    Z=norm.ppf(cr)
    df['Adjusted Population'] = df['Population Mean'] + (Z * df['Population SD'])
    return df

def display_ca_eoq(cr, days):
    # calculate the hospitalization EOQ
    epi = calc_eoq(ca_epi_df, cr)
    eoq_df = 0.243 * epi
    # adjusting the non-COVID patients
    eoq_sheet = to_sheet(eoq_df)
    display_population(eoq_sheet, round=True)
    # calculate stockpile projections
    preds_df = Data(
        "demand_by_pop_total_pn_tc", config)
    preds_df.array = (model.demand.demand_by_pop_per_person_pn_uc.array.T * ca_epi_df["Adjusted Population"].to_numpy().T).T
    preds_df.array *= days
    preds_df.df.drop(['Ventilators'], axis=1, inplace=True)
    preds_sheet = to_sheet(preds_df.df)
    chart = util.generate_group_bar_legend(preds_df.df, scientific_notation=True)
    display_population(preds_sheet, round=True)
    display(chart)
    
ca_burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))

ca_cr_slider = widgets.FloatSlider(min=0.70,max=0.99,step=0.01,value=0.95, continuous_update=False, description="CR")
ca_day_slider = widgets.IntSlider(min=1, max=120, value=30, continuous_update=False, description="Days")
ca_out = widgets.interactive_output(display_ca_eoq, {'cr': ca_cr_slider, 'days': ca_day_slider})

In [5]:
widgets.VBox([ca_cr_slider, ca_day_slider, ca_out, burn_chart])

## OES Model
Populations gathered from Bureau of Labor Statistics May 2019 reporting. Stockpile projections calculated using Washington burn rate assumptions, shown below. Individual occupations (denoted by SOC codes) are mapped to one of 7 protection levels, where `WA0` represents a hypothetical person who requires no protection, and `WA6` a front-line healthcare worker with frequent exposure to COVID-19.

In [6]:
oes_restart = RestartModel(
    population='oes',
    state='California',
    subpop='healthcare')
oes_model = oes_restart.model

oes_burn_df = oes_model.demand.demand_per_unit_map_dn_um.df
oes_burn_df.index.name = "Population"
oes_burn_chart = util.generate_group_bar_legend(oes_model.demand.demand_per_unit_map_dn_um.df)

oes_burn_rates = qgrid.show_grid(oes_burn_df)

In [7]:
display(create_run_button("Run Model"))
widgets.VBox([oes_burn_rates])

Button(description='Run Model', style=ButtonStyle())

In [8]:
oes_model.demand.adjust_burn(oes_burn_rates.get_changed_df().to_numpy())
oes_slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    df_chart = df
    df_chart.index = df_chart.index.get_level_values(1)
    chart = util.generate_group_bar_legend(df_chart, scientific_notation=True)
    display(sheet)
    display(chart)
    
def set_stock(backstop):
    oes_model.inventory.set_average_orders_per_period(oes_model.demand.demand_by_popsum1_total_rp1n_tc)
    backstop = [backstop]
    oes_model.inventory.order(oes_model.inventory.inv_by_popsum1_total_rp1n_tc)
    oes_model.inventory.set_min_in_periods(backstop)
    display_stock(oes_model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
oes_pop = format_population(to_sheet(oes_model.population.population_pP_tr.df))
oes_out = widgets.interactive_output(dashboard, {'backstop': oes_slider})

widgets.VBox([oes_slider, oes_out, oes_burn_chart])

## JHU Model
PPE forecasting using Calcat population assumptions and burn rate assumptions obtained from Johns Hopkins University.

In [9]:
jhu_config = set_config(os.path.join(PATH, 'config/jhu'))
jhu_restart = RestartModel(config='jhu', population='dict')
jhu_model = jhu_restart.model

jhu_burn_df = jhu_model.demand.demand_per_unit_map_dn_um.df
jhu_burn_df.index.name = "Population"
jhu_burn_chart = util.generate_group_bar_legend(jhu_model.demand.demand_per_unit_map_dn_um.df)

jhu_burn_rates = qgrid.show_grid(jhu_burn_df)

In [10]:
display(create_run_button("Run Model"))
widgets.VBox([jhu_burn_rates])

Button(description='Run Model', style=ButtonStyle())

In [11]:
jhu_model.demand.adjust_burn(jhu_burn_rates.get_changed_df().to_numpy())

jhu_epi_df = data.epi_ranges()
jhu_epi_df.set_index('Model', drop=True, inplace=True)
jhu_epi_df.drop(['CA Projected Non-COVID Patients'], axis=0, inplace=True)

# adjusting so it's in terms of population, since we're not doing vents here
jhu_epi_df = jhu_epi_df * 4.12
jhu_epi_df.columns = ['Population Low', 'Population Mid', 'Population High']
jhu_stdev = jhu_epi_df.apply(
    lambda row: util.triangular(
        row['Population Mid'],
        row['Population Low'], 
        row['Population High']
    ), axis=1
)

jhu_epi_df['Population Mean'] = jhu_epi_df.mean(axis=1)

jhu_epi_df['Population SD'] = jhu_stdev


def calc_eoq(df, cr):
    Z=norm.ppf(cr)
    df['Adjusted Population'] = df['Population Mean'] + (Z * df['Population SD'])
    return df

def display_jhu_eoq(cr, days):
    # calculate the hospitalization EOQ
    epi = calc_eoq(jhu_epi_df, cr)
    eoq_df = 0.243 * epi
    # adjusting the non-COVID patients
    eoq_sheet = to_sheet(eoq_df)
    display_population(eoq_sheet, round=True)
    # calculate stockpile projections
    preds_df = Data(
        "demand_by_pop_total_pn_tc", jhu_config)
    preds_df.array = (jhu_model.demand.demand_by_pop_per_person_pn_uc.array.T * jhu_epi_df["Adjusted Population"].to_numpy().T).T
    preds_df.array *= days
    preds_df.df.rename({'Glove Pairs': 'Gloves'}, axis=1, inplace=True)
    preds_sheet = to_sheet(preds_df.df)
    chart = util.generate_group_bar_legend(preds_df.df, scientific_notation=True)
    display_population(preds_sheet, round=True)
    display(chart)
    
jhu_burn_sheet = format_population(to_sheet(jhu_model.demand.demand_per_unit_map_dn_um.df))

jhu_cr_slider = widgets.FloatSlider(min=0.70,max=0.99,step=0.01,value=0.95, continuous_update=False, description="CR")
jhu_day_slider = widgets.IntSlider(min=1, max=120, value=30, continuous_update=False, description="Days")
jhu_out = widgets.interactive_output(display_jhu_eoq, {'cr': jhu_cr_slider, 'days': jhu_day_slider})
widgets.VBox([jhu_cr_slider, jhu_day_slider, jhu_out, jhu_burn_chart])

## Burn Rate Visualization
Edit these tables to adjust assumptions, and scroll down for a detailed per-resource breakdown of burn rate across each model. We provide both bar graphs and a scatterplot view.

In [12]:
jhu_model.demand.demand_per_unit_map_dn_um.df.index.name = "Population"
oes_model.demand.demand_per_unit_map_dn_um.df.index.name = "Population"
model.demand.demand_per_unit_map_dn_um.df.index.name = "Population"

jhu_dem = qgrid.show_grid(jhu_model.demand.demand_per_unit_map_dn_um.df)
oes_dem = qgrid.show_grid(oes_model.demand.demand_per_unit_map_dn_um.df)
ca_dem = qgrid.show_grid(model.demand.demand_per_unit_map_dn_um.df)

display(create_run_button("Update Burn Rates"))
widgets.VBox([jhu_dem, oes_dem, ca_dem])

Button(description='Update Burn Rates', style=ButtonStyle())

In [13]:
jhu_model.demand.adjust_burn(jhu_dem.get_changed_df().to_numpy())
oes_model.demand.adjust_burn(oes_dem.get_changed_df().to_numpy())
model.demand.adjust_burn(ca_dem.get_changed_df().to_numpy())

# rename so that resource names are consistent
jhu_model.demand.demand_per_unit_map_dn_um.df.rename(
    {'Glove Pairs': 'Gloves', 'Simple Masks': 'Non ASTM Mask'}, axis=1, inplace=True
)
model.demand.demand_per_unit_map_dn_um.df.rename(
    {'Surgical Masks': 'ASTM 3 Mask'}, axis=1, inplace=True
)

oes_model.demand.demand_per_unit_map_dn_um.df['N95 Masks'] = (
    oes_model.demand.demand_per_unit_map_dn_um.df['N95 Surgical'] + oes_model.demand.demand_per_unit_map_dn_um.df['N95 Non-surgical']
)

oes_model.demand.demand_per_unit_map_dn_um.df.drop(['N95 Surgical', 'N95 Non-surgical'], axis=1, inplace=True)
model.demand.demand_per_unit_map_dn_um.df.drop(['Ventilators', 'PAPR', 'Coveralls'], axis=1, inplace=True)

burn_summary_df = pd.concat([
    jhu_model.demand.demand_per_unit_map_dn_um.df,
    oes_model.demand.demand_per_unit_map_dn_um.df,
    model.demand.demand_per_unit_map_dn_um.df
])

burn_summary_df.fillna(0, inplace=True)
burn_summary_df.index.name = "Model"

burn_summary_chart_list = util.generate_separate_bar_list(burn_summary_df)
burn_summary_chart_legend = util.generate_html_legend(burn_summary_df)

burn_summary_scatter_list = util.generate_separate_scatter_list(burn_summary_df)

In [14]:
display(burn_summary_chart_list[0], burn_summary_scatter_list[0], burn_summary_chart_legend)

Figure(axes=[Axis(scale=OrdinalScale(), tick_style={'font-size': '6'}), Axis(orientation='vertical', scale=Lin…

Figure(axes=[Axis(scale=OrdinalScale(), tick_style={'font-size': '6'}), Axis(orientation='vertical', scale=Lin…

HTML(value="<div style='font-size:13px; font-family:helvetica'><b style='font-weight:bold'>Model</b><table><tr…

In [15]:
display(burn_summary_chart_list[1], burn_summary_scatter_list[1], burn_summary_chart_legend)

Figure(axes=[Axis(scale=OrdinalScale(), tick_style={'font-size': '6'}), Axis(orientation='vertical', scale=Lin…

Figure(axes=[Axis(scale=OrdinalScale(), tick_style={'font-size': '6'}), Axis(orientation='vertical', scale=Lin…

HTML(value="<div style='font-size:13px; font-family:helvetica'><b style='font-weight:bold'>Model</b><table><tr…

In [16]:
display(burn_summary_chart_list[2], burn_summary_scatter_list[2], burn_summary_chart_legend)

Figure(axes=[Axis(scale=OrdinalScale(), tick_style={'font-size': '6'}), Axis(orientation='vertical', scale=Lin…

Figure(axes=[Axis(scale=OrdinalScale(), tick_style={'font-size': '6'}), Axis(orientation='vertical', scale=Lin…

HTML(value="<div style='font-size:13px; font-family:helvetica'><b style='font-weight:bold'>Model</b><table><tr…

In [17]:
display(burn_summary_chart_list[3], burn_summary_scatter_list[3], burn_summary_chart_legend)

Figure(axes=[Axis(scale=OrdinalScale(), tick_style={'font-size': '6'}), Axis(orientation='vertical', scale=Lin…

Figure(axes=[Axis(scale=OrdinalScale(), tick_style={'font-size': '6'}), Axis(orientation='vertical', scale=Lin…

HTML(value="<div style='font-size:13px; font-family:helvetica'><b style='font-weight:bold'>Model</b><table><tr…

In [18]:
display(burn_summary_chart_list[5], burn_summary_scatter_list[5], burn_summary_chart_legend)

Figure(axes=[Axis(scale=OrdinalScale()), Axis(orientation='vertical', scale=LinearScale(), tick_format='.1f')]…

Figure(axes=[Axis(scale=OrdinalScale()), Axis(orientation='vertical', scale=LinearScale(), tick_format='.1f')]…

HTML(value="<div style='font-size:13px; font-family:helvetica'><b style='font-weight:bold'>Model</b><table><tr…

In [19]:
display(burn_summary_chart_list[6], burn_summary_scatter_list[6], burn_summary_chart_legend)

Figure(axes=[Axis(scale=OrdinalScale(), tick_style={'font-size': '6'}), Axis(orientation='vertical', scale=Lin…

Figure(axes=[Axis(scale=OrdinalScale(), tick_style={'font-size': '6'}), Axis(orientation='vertical', scale=Lin…

HTML(value="<div style='font-size:13px; font-family:helvetica'><b style='font-weight:bold'>Model</b><table><tr…

In [20]:
display(burn_summary_chart_list[7], burn_summary_scatter_list[7], burn_summary_chart_legend)

Figure(axes=[Axis(scale=OrdinalScale()), Axis(orientation='vertical', scale=LinearScale(), tick_format='.1f')]…

Figure(axes=[Axis(scale=OrdinalScale()), Axis(orientation='vertical', scale=LinearScale(), tick_format='.1f')]…

HTML(value="<div style='font-size:13px; font-family:helvetica'><b style='font-weight:bold'>Model</b><table><tr…